In [1]:
import numpy as np
import pandas as pd
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.colors as clrs
plt.rcParams.update({"text.usetex": True})

import math as mt
import energyflow as ef

import modplot

## Load Datasets

In [ ]:
# use a fraction of the full datasets (amount=1.0 uses the full datasets)
amount =0.01

# select jets with JEC-corrected pT in [375, 425] GeV and |eta| < 1.9, with "medium" quality
specs = ['375 <= corr_jet_pts <= 425', 'abs_jet_eta < 1.9', 'quality >= 2']

# stamp description of the jet selections
legend_label_0 = r'AK5 Jets, $|\eta^{\rm jet}|<1.9$'
legend_label_1 = r'$p_T^{\rm jet}\in [375, 425]$ GeV'

# load the CMS (cms), Pythia-generated (gen), and detector-simulated (sim) datasets
cms = ef.mod.load(*specs, dataset='cms', amount=amount, validate_files=True)
sim = ef.mod.load(*specs, dataset='sim', amount=amount, validate_files=True, store_gens=False)
gen = ef.mod.load(*specs, dataset='gen', amount=amount, validate_files=True)

# show the details of the CMS, SIM, and GEN datasets
print(cms)
print(sim)
print(gen)

### Define a function that calculates $\Delta\phi$ and handles periodic boundary conditions $[0, 2\pi] \rightarrow [-\pi,\pi]$

In [ ]:
def deltaPhi( p1, p2):
#Computes delta phi, handling periodic limit conditions.
    res = p1 - p2
    while res > mt.pi:
        res -= 2*mt.pi
    while res < -mt.pi:
        res += 2*mt.pi
    return res 

## CMS Data

##### Add original indexes as a column to be able to later match jets with respective pfcs. 
###### Below, the final dataframe, contains ::::::::         jets_i + original index position + jets_f   arrays

In [ ]:
# Create the Dataframe
cms_i_df = pd.DataFrame(cms.jets_i, columns=cms.jets_i_cols)

# Create a sample of index positions [0, 1, 2, 3, 4, .....,N-1 ,N]
col0 = np.linspace(0, len(cms.jets_i), num=len(cms.jets_i), dtype=int, endpoint=False)

# Insert those indexes into the Dataframe 
# Indexes are inserted after the final column of the original cms_df Dataframe
end = len(cms.jets_i[0])
cms_i_df.insert(end, 'index_pos', col0)

# Create a Dataframe for jets_f array
cms_f_df = pd.DataFrame(cms.jets_f, columns=cms.jets_f_cols)

# Concatenate the 2 Dataframes
cms_df = pd.concat([cms_i_df, cms_f_df], axis=1)

# Show the final Dataframe
cms_df

### Find duplicate rows

In [ ]:
# Find the duplicates and sort based also on jet_pt
key = cms_df.duplicated(subset=['rn', 'lbn', 'evn'], keep=False)
cms_multijetdf = cms_df[key]
cms_multijetdf = cms_multijetdf.sort_values(by=['rn', 'lbn', 'evn','jet_pt'], axis=0, ascending=[True, True, True, False])
cms_multijetdf

###################################################

# If u want to check result, remove comment from below
#y = cms_df.value_counts(subset=['rn','lbn', 'evn']).to_frame(name='count').reset_index()
#y.loc[y['count'] == 2]
#y
####################################################

## Sim Data

In [ ]:
# Create the Dataframe
sim_i_df = pd.DataFrame(sim.jets_i, columns=sim.jets_i_cols)

# Create a sample of index positions [0, 1, 2, 3, 4, .....,N-1 ,N]
col0 = np.linspace(0, len(sim.jets_i), num=len(sim.jets_i), dtype=int, endpoint=False)

# Insert those indexes into the Dataframe 
# Indexes are inserted after the final column of the original sim_df Dataframe
end = len(sim.jets_i[0])
sim_i_df.insert(end, 'index_pos', col0)

# Create a Dataframe for jets_f array
sim_f_df = pd.DataFrame(sim.jets_f, columns=sim.jets_f_cols)

# Concatenate the 2 Dataframes
sim_df = pd.concat([sim_i_df, sim_f_df], axis=1)

# Show the final Dataframe
sim_df

In [ ]:
# Find the duplicates and sort based also on jet_pt
key = sim_df.duplicated(subset=['rn', 'lbn', 'evn'], keep=False)
sim_multijetdf = sim_df[key]
sim_multijetdf = sim_multijetdf.sort_values(by=['rn', 'lbn', 'evn','jet_pt'], axis=0, ascending=[True, True, True, False])
sim_multijetdf

###################################################

# If u want to check result, remove comment from below

#y = sim_multijetdf.value_counts(subset=['rn','lbn', 'evn']).to_frame(name='count').reset_index()
#y.loc[y['count'] == 2]
#y
####################################################

## Gen Data

In [ ]:
# Create the Dataframe
gen_i_df = pd.DataFrame(gen.jets_i, columns=gen.jets_i_cols)

# Create a sample of index positions [0, 1, 2, 3, 4, .....,N-1 ,N]
col0 = np.linspace(0, len(gen.jets_i), num=len(gen.jets_i), dtype=int, endpoint=False)

# Insert those indexes into the Dataframe 
# Indexes are inserted after the final column of the original gen_df Dataframe
end = len(gen.jets_i[0])
gen_i_df.insert(end, 'index_pos', col0)

# Create a Dataframe for jets_f array
gen_f_df = pd.DataFrame(gen.jets_f, columns=gen.jets_f_cols)

# Concatenate the 2 Dataframes
gen_df = pd.concat([gen_i_df, gen_f_df], axis=1)

# Show the final Dataframe
gen_df

In [ ]:
# Find the duplicates and sort based also on jet_pt
key = gen_df.duplicated(subset=['rn', 'lbn', 'evn'], keep=False)
gen_multijetdf = gen_df[key]
gen_multijetdf = gen_multijetdf.sort_values(by=['rn', 'lbn', 'evn','jet_pt'], axis=0, ascending=[True, True, True, False])
gen_multijetdf

###################################################

# If u want to check result, remove comment from below
#y = gen_multijetdf.value_counts(subset=['rn','lbn', 'evn']).to_frame(name='count').reset_index()
#y.loc[y['count'] == 1]
#y
####################################################

## Make those histogramms after all

First turn dataframes into arrays

In [ ]:
# Cms, Sim and Gen all-inclusive array

cms_allarr = np.asarray(cms_df)
sim_allarr = np.asarray(sim_df)
gen_allarr = np.asarray(gen_df)


# Cms, Sim and Gen multijet array

cms_multijet_arr = np.asarray(cms_multijetdf)
sim_multijet_arr = np.asarray(sim_multijetdf)
gen_multijet_arr = np.asarray(gen_multijetdf)

### 1D all-inclusive jet_pt histogramm

In [ ]:
# This chuck of code belongs to mr P.Komiske
# function for getting histograms from observable values
def calc_hist(vals, log=False, bins=10, weights=None, density=True):
    
    if weights is None:
        weights = np.ones(vals.shape)
    
    # compute histogram
    hist, bins= np.histogram(vals, bins=bins, weights=weights)
    
    if log==True:
        plt.semilogy()
    # compute which bins the values are in
    digits = np.digitize(vals, bins)

    # compute the errors per bin
    # note that lowest bin value that digitize returns is 1
    # hence the range in the following list comprehension should start at 1
    errs = np.asarray([np.linalg.norm(weights[digits==i]) for i in range(1, len(bins))])
    #print(len(bins))
    #print(errs)
    # handle normalization
    if density:
        density_int = weights.sum() * (bins[1] - bins[0])
        hist /= density_int
        errs /= density_int
        
    return hist, errs, bins

### Compute histogramm

In [ ]:
# Printout of columns that exist in array

# Cms
print(cms_df.columns)

# Sim
print(sim_df.columns)

# Gen
print(gen_df.columns)

In [ ]:
# define your histogram binning
alpha = gen_df['jet_pt'].min()
omega = gen_df['jet_pt'].max()
bins = np.linspace(alpha, omega, 40)

# get middle bin values and step size
step = bins[1]-bins[0]
midbins = (bins[1:] + bins[:-1])/2

# which column to hist

col0 = 7
col1 = 8
col2 = 6


# cms histogram calculation
cms_hist, cms_errs, _ = calc_hist(cms_allarr[:, col0]*cms_allarr[:, 12], bins=bins, weights=None, density=True)
cms_hist, cms_errs = cms_hist/step, cms_errs/step

# sim histogram calculation
sim_hist, sim_errs, _ = calc_hist(sim_allarr[:, col1]*sim_allarr[:, 13], bins=bins, weights=None, density=True)
sim_hist, sim_errs = sim_hist/step, sim_errs/step

# gen histogram calculation
gen_hist, gen_errs, _ = calc_hist(gen_allarr[:, col2], bins=bins, weights=None, density=True)
gen_hist, gen_errs = gen_hist/step, gen_errs/step


### Make dem plots

In [ ]:
############################
# gridspec & figsize options
gridspec_kw = {'height_ratios': (3.5, 1), 'hspace': 0.0}
figsize = (6, 6)

##############
# get subplots
nsubplots = 2
fig, axes = plt.subplots(nrows=nsubplots, gridspec_kw=gridspec_kw, figsize=figsize)

#############
# axes limits
xlim=(min(bins), max(bins))
ylim=(0, 1.5*np.max(cms_hist))
ylim_ratio=(0.25, 1.75)
      
for ax in axes:
    ax.set_xlim(xlim)

axes[0].set_ylim(ylim)
axes[1].set_ylim(ylim_ratio)  

##########################################
# define the x-axis label and y-axis label
xlabel = r'Jet pt '
ylabel = 'Probability density'
units='GeV'
ylabel_ratio='Ratio to\nsim'

xlabel = r'{} [{}]'.format(xlabel, units)
ylabel = r'{} [{}{}]'.format(ylabel, units, r'$^{-1}$')
axes[-1].set_xlabel(xlabel)
axes[-1].set_ylabel(ylabel_ratio, fontsize=8)
axes[0].set_ylabel(ylabel)

###############
# Tick settings
for ax in axes:
    ax.minorticks_on()
    ax.tick_params(top=True, right=True, bottom=True, left=True, direction='in', which='both')

axes[0].tick_params(labelbottom=False)
axes[1].tick_params(axis='y', labelsize=8)

################################################################################################
################################################################################################

# tiny regulator
reg = 10**-30

# plot of the cms observable distribution
cms_style = {'lw':1.5, 'capsize':1., 'capthick':1, 'markersize':1., 'color':'black', 
             'label':'CMS 2011 Open Data'}
axes[0].errorbar(midbins, cms_hist, xerr=step/2, yerr=cms_errs, **modplot.cms_style(), zorder=3)
axes[1].errorbar(midbins, cms_hist/(sim_hist + reg), xerr=step/2, yerr=cms_errs/(sim_hist+reg), 
                **cms_style, zorder=2)

# plot of the sim observable distribution
sim_style = {'lw':1.5, 'capsize':1., 'capthick':1, 'markersize':1., 'color':'orange', 
             'label':'CMS 2011 Simulation'}
axes[0].errorbar(midbins, sim_hist, xerr=step/2, yerr=sim_errs, **modplot.sim_style(), zorder=2)
axes[1].errorbar(midbins, sim_hist/(sim_hist+reg), xerr=step/2, yerr=sim_errs/(sim_hist+reg), 
                **sim_style, zorder=1)

# plot of the gen observable distribution
gen_style = {'lw': 1.5, 'ls': '--', 'color': 'blue', 'label': r'\textsc{Pythia 6} Generation'}
axes[0].plot(midbins, gen_hist, **modplot.gen_style(), zorder=0)
axes[1].plot(midbins, gen_hist/(sim_hist + reg), **gen_style, zorder=0)

# additional plot modifications

stamp_x, stamp_y = 0.70, 0.70
modplot.stamp(stamp_x, stamp_y, ax=axes[0], 
              line_0=legend_label_0, line_1=legend_label_1)
modplot.legend(ax=axes[0], order=[1, 2, 0])

# save plot (by default in the same directory as this notebook).
# The MOD logo can be optionally added by changing add_watermark to True in the following command.
modplot.save(fig, 'All-jet_pt-jec', add_watermark=False, tx=51.5, ty=251.5, plots_dir='.')


fig.show()

# Print min and max value of histogramm

print(f'''Min of the hist is :\t {alpha}
        \nMax of the hist is : \t {omega}''')

## Hardest jet pt hist

Dataframe already sorted. For duplicate lines, 1st occurence includes hard jet 

In [ ]:
# For Cms
cms_hardjet = np.asarray(cms_multijetdf.drop_duplicates(subset=['rn', 'lbn', 'evn'], keep='first'))
cms_softjet = np.asarray(cms_multijetdf.drop_duplicates(subset=['rn', 'lbn', 'evn'], keep='last'))

# For Sim
sim_hardjet = np.asarray(sim_multijetdf.drop_duplicates(subset=['rn', 'lbn', 'evn'], keep='first'))
sim_softjet = np.asarray(sim_multijetdf.drop_duplicates(subset=['rn', 'lbn', 'evn'], keep='last'))

# For Gen
gen_hardjet = np.asarray(gen_multijetdf.drop_duplicates(subset=['rn', 'lbn', 'evn'], keep='first'))
gen_softjet = np.asarray(gen_multijetdf.drop_duplicates(subset=['rn', 'lbn', 'evn'], keep='last'))

In [ ]:
# Printout of columns that exist in array

# Cms
print(cms_multijetdf.columns)

# Sim
print(sim_multijetdf.columns)

# Gen
print(gen_multijetdf.columns)

## Histogramms

In [ ]:
# define your histogram binning
alpha = gen_df['jet_pt'].min()
omega = gen_df['jet_pt'].max()
bins = np.linspace(alpha, omega, 40)

# get middle bin values and step size
step = bins[1]-bins[0]
midbins = (bins[1:] + bins[:-1])/2

# which column to hist

col0 = 7
col1 = 8
col2 = 6


# cms histogram calculation
cms_hist, cms_errs, _ = calc_hist(cms_hardjet[:, col0]*cms_hardjet[:, 12], bins=bins, weights=None, density=True)
cms_hist, cms_errs = cms_hist/step, cms_errs/step

# sim histogram calculation
sim_hist, sim_errs, _ = calc_hist(sim_hardjet[:, col1]*sim_hardjet[:, 13], bins=bins, weights=None, density=True)
sim_hist, sim_errs = sim_hist/step, sim_errs/step

# gen histogram calculation
gen_hist, gen_errs, _ = calc_hist(gen_hardjet[:, col2], bins=bins, weights=None, density=True)
gen_hist, gen_errs = gen_hist/step, gen_errs/step

In [ ]:
############################
# gridspec & figsize options
gridspec_kw = {'height_ratios': (3.5, 1), 'hspace': 0.0}
figsize = (7, 7)

##############
# get subplots
nsubplots = 2
fig, axes = plt.subplots(nrows=nsubplots, gridspec_kw=gridspec_kw, figsize=figsize)

#############
# axes limits
xlim=(min(bins), max(bins))
ylim=(0, 1.5*np.max(cms_hist))
ylim_ratio=(0.25, 1.75)
      
for ax in axes:
    ax.set_xlim(xlim)

axes[0].set_ylim(ylim)
axes[1].set_ylim(ylim_ratio)  

##########################################
# define the x-axis label and y-axis label & Titel
titel = 'Hardest jet pt'
xlabel = r'Jet pt '
ylabel = 'Probability density'
units='GeV'
ylabel_ratio='Ratio to\nsim'

xlabel = r'{} [{}]'.format(xlabel, units)
ylabel = r'{} [{}{}]'.format(ylabel, units, r'$^{-1}$')
axes[-1].set_xlabel(xlabel)
axes[-1].set_ylabel(ylabel_ratio, fontsize=8)
axes[0].set_ylabel(ylabel)

###############
# Tick settings
for ax in axes:
    ax.minorticks_on()
    ax.tick_params(top=True, right=True, bottom=True, left=True, direction='in', which='both')

axes[0].tick_params(labelbottom=False)
axes[1].tick_params(axis='y', labelsize=8)

################################################################################################
################################################################################################

# tiny regulator
reg = 10**-30

# plot of the cms observable distribution
cms_style = {'lw':1.5, 'capsize':1., 'capthick':1, 'markersize':1., 'color':'black', 
             'label':'CMS 2011 Open Data'}
axes[0].errorbar(midbins, cms_hist, xerr=step/2, yerr=cms_errs, **modplot.cms_style(), zorder=3)
axes[1].errorbar(midbins, cms_hist/(sim_hist + reg), xerr=step/2, yerr=cms_errs/(sim_hist+reg), 
                **cms_style, zorder=2)

# plot of the sim observable distribution
sim_style = {'lw':1.5, 'capsize':1., 'capthick':1, 'markersize':1., 'color':'orange', 
             'label':'CMS 2011 Simulation'}
axes[0].errorbar(midbins, sim_hist, xerr=step/2, yerr=sim_errs, **modplot.sim_style(), zorder=2)
axes[1].errorbar(midbins, sim_hist/(sim_hist+reg), xerr=step/2, yerr=sim_errs/(sim_hist+reg), 
                **sim_style, zorder=1)

# plot of the gen observable distribution
gen_style = {'lw': 1.5, 'ls': '--', 'color': 'blue', 'label': r'\textsc{Pythia 6} Generation'}
axes[0].plot(midbins, gen_hist, **modplot.gen_style(), zorder=0)
axes[1].plot(midbins, gen_hist/(sim_hist + reg), **gen_style, zorder=0)

# additional plot modifications

stamp_x, stamp_y = 0.70, 0.70
modplot.stamp(stamp_x, stamp_y, ax=axes[0], 
              line_0=legend_label_0, line_1=legend_label_1)
modplot.legend(ax=axes[0], order=[1, 2, 0])

# save plot (by default in the same directory as this notebook).
# The MOD logo can be optionally added by changing add_watermark to True in the following command.
modplot.save(fig, 'Hard-jet_pt-jec', add_watermark=False, tx=51.5, ty=251.5, plots_dir='.')

fig.suptitle(titel, fontsize=16)
fig.show()

# Print min and max value of histogramm

print(f'''Min of the hist is :\t {alpha}
        \nMax of the hist is : \t {omega}''')

## Softer jet pt hist

In [ ]:
# define your histogram binning
alpha = gen_df['jet_pt'].min()
omega = gen_df['jet_pt'].max()
bins = np.linspace(alpha, omega, 40)

# get middle bin values and step size
step = bins[1]-bins[0]
midbins = (bins[1:] + bins[:-1])/2

# which column to hist

col0 = 7
col1 = 8
col2 = 6


# cms histogram calculation
cms_hist, cms_errs, _ = calc_hist(cms_softjet[:, col0]*cms_softjet[:, 12], bins=bins, weights=None, density=True)
cms_hist, cms_errs = cms_hist/step, cms_errs/step

# sim histogram calculation
sim_hist, sim_errs, _ = calc_hist(sim_softjet[:, col1]*sim_softjet[:, 13], bins=bins, weights=None, density=True)
sim_hist, sim_errs = sim_hist/step, sim_errs/step

# gen histogram calculation
gen_hist, gen_errs, _ = calc_hist(gen_softjet[:, col2], bins=bins, weights=None, density=True)
gen_hist, gen_errs = gen_hist/step, gen_errs/step

In [ ]:
############################
# gridspec & figsize options
gridspec_kw = {'height_ratios': (3.5, 1), 'hspace': 0.0}
figsize = (6, 6)

##############
# get subplots
nsubplots = 2
fig, axes = plt.subplots(nrows=nsubplots, gridspec_kw=gridspec_kw, figsize=figsize)

#############
# axes limits
xlim=(min(bins), max(bins))
ylim=(0, 1.5*np.max(cms_hist))
ylim_ratio=(0.25, 1.75)
      
for ax in axes:
    ax.set_xlim(xlim)

axes[0].set_ylim(ylim)
axes[1].set_ylim(ylim_ratio)  

##########################################
# define the x-axis label and y-axis label & Titel
titel = 'Soft jet pt'
xlabel = r'Jet pt '
ylabel = 'Probability density'
units='GeV'
ylabel_ratio='Ratio to\nsim'

xlabel = r'{} [{}]'.format(xlabel, units)
ylabel = r'{} [{}{}]'.format(ylabel, units, r'$^{-1}$')
axes[-1].set_xlabel(xlabel)
axes[-1].set_ylabel(ylabel_ratio, fontsize=8)
axes[0].set_ylabel(ylabel)

###############
# Tick settings
for ax in axes:
    ax.minorticks_on()
    ax.tick_params(top=True, right=True, bottom=True, left=True, direction='in', which='both')

axes[0].tick_params(labelbottom=False)
axes[1].tick_params(axis='y', labelsize=8)

################################################################################################
################################################################################################

# tiny regulator
reg = 10**-30

# plot of the cms observable distribution
cms_style = {'lw':1.5, 'capsize':1., 'capthick':1, 'markersize':1., 'color':'black', 
             'label':'CMS 2011 Open Data'}
axes[0].errorbar(midbins, cms_hist, xerr=step/2, yerr=cms_errs, **modplot.cms_style(), zorder=3)
axes[1].errorbar(midbins, cms_hist/(sim_hist + reg), xerr=step/2, yerr=cms_errs/(sim_hist+reg), 
                **cms_style, zorder=2)

# plot of the sim observable distribution
sim_style = {'lw':1.5, 'capsize':1., 'capthick':1, 'markersize':1., 'color':'orange', 
             'label':'CMS 2011 Simulation'}
axes[0].errorbar(midbins, sim_hist, xerr=step/2, yerr=sim_errs, **modplot.sim_style(), zorder=2)
axes[1].errorbar(midbins, sim_hist/(sim_hist+reg), xerr=step/2, yerr=sim_errs/(sim_hist+reg), 
                **sim_style, zorder=1)

# plot of the gen observable distribution
gen_style = {'lw': 1.5, 'ls': '--', 'color': 'blue', 'label': r'\textsc{Pythia 6} Generation'}
axes[0].plot(midbins, gen_hist, **modplot.gen_style(), zorder=0)
axes[1].plot(midbins, gen_hist/(sim_hist + reg), **gen_style, zorder=0)

# additional plot modifications

stamp_x, stamp_y = 0.70, 0.70
modplot.stamp(stamp_x, stamp_y, ax=axes[0], 
              line_0=legend_label_0, line_1=legend_label_1)
modplot.legend(ax=axes[0], order=[1, 2, 0])

# save plot (by default in the same directory as this notebook).
# The MOD logo can be optionally added by changing add_watermark to True in the following command.
modplot.save(fig, 'Soft-jet_pt-jec', add_watermark=False, tx=51.5, ty=251.5, plots_dir='.')

fig.suptitle(titel, fontsize=14)
fig.show()

# Print min and max value of histogramm

print(f'''Min of the hist is :\t {alpha}
        \nMax of the hist is : \t {omega}''')